# Time-Series Generation using Contrastive Learning

Consider learning a generative model for time-series data.

The sequential setting poses a unique challenge: Not only should the generator capture the conditional dynamics of (stepwise) transitions, but its open-loop rollouts should also preserve the joint distribution of (multi-step) trajectories.

On one hand, autoregressive models
trained by MLE allow learning and computing explicit transition distributions, but suffer from compounding error during rollouts.

On the other hand, adversarial models based on GAN training alleviate such exposure bias, but transitions are implicit and hard to assess.

In this work, we study a generative framework that seeks to combine the strengths of both: Motivated by a moment-matching objective to mitigate
compounding error, we optimize a local (but forward-looking) *transition policy*, where the reinforcement signal is provided by a global (but stepwise-decomposable) *energy model* trained by contrastive estimation. 

At **training**, the two components are learned cooperatively, avoiding the instabilities typical of adversarial objectives. 

At **inference**, the learned policy serves as the generator for iterative sampling, and the learned energy serves as a trajectory-level measure for evaluating sample quality.

By expressly training a policy to imitate sequential behavior of time-series features in a dataset, this approach embodies *“generation by imitation”*. Theoretically, we illustrate the correctness of this formulation and the consistency of the algorithm.

Empirically, we evaluate its ability to generate predictively useful samples from real-world datasets, verifying that it performs at the standard of existing benchmarks.

## 1 Setup

### 1.1 Install libraries

Run the cell below to **install** the necessary libraries.

In [ ]:
%pip install wandb
%pip install pytorch-lightning
%pip install plotly
%pip install tensorflow

### 1.2 Import Libraries

Run the cell below to **import** the necessary libraries

In [ ]:
import random
import numpy as np
import torch

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

In [ ]:
import wandb
from pytorch_lightning.loggers.wandb import WandbLogger

Eh eh

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### 1.3 Hyper-parameters

The cell below contains *all* the hyper-parameters nedded by this script, for easy tweaking.

In [ ]:
from hyperparamets import Config
hparams = Config()

Comment this cell if you don't want to use Weights & Biases to log the process

In [ ]:
#!wandb login

### 1.5 Initialization

Initialize the modules needed by running the cells in this section.

#### 1.5.1 reproducibility.

In [ ]:
np.random.seed(0)
random.seed(0)

torch.cuda.manual_seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True # Can have performance impact
torch.backends.cudnn.benchmark = False

_ = pl.seed_everything(0)

#### 1.5.2 Device

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}.")

#### 1.5.3 Data

Generate the dataset as requested

In [ ]:
import os
from hyperparamets import Config
print(os.listdir('.'))
hparams = Config()

In [ ]:
from data_generation import iid_sequence_generator, sine_process, wiener_process

In [ ]:
if hparams.dataset_name in ['sine', 'wien', 'iid', 'cov']:
  # Generate and store the dataset as requested
  dataset_path = f"../datasets/{hparams.dataset_name}_generated_stream.csv"
  if hparams.dataset_name == 'sine':
    sine_process.save_sine_process(p=hparams.data_dim, N=hparams.num_samples, file_path=dataset_path)
  elif hparams.dataset_name == 'wien':
    wiener_process.save_wiener_process(p=hparams.data_dim, N=hparams.num_samples, file_path=dataset_path)
    print("\n")
  elif hparams.dataset_name == 'iid':
    iid_sequence_generator.save_iid_sequence(p=hparams.data_dim, N=hparams.num_samples, file_path=dataset_path)
  elif hparams.dataset_name == 'cov':
    iid_sequence_generator.save_cov_sequence(p=hparams.data_dim, N=hparams.num_samples, file_path=dataset_path)
  else:
    raise ValueError
  print(f"The {hparams.dataset_name} dataset has been succesfully created and stored into:\n\t- {dataset_path}")
elif hparams.dataset_name == 'real':
  pass
else:
  raise ValueError("Dataset not supported.")

Train / Test split

In [ ]:
from dataset_handling import train_test_split

In [ ]:
if hparams.dataset_name in ['sine', 'wien', 'iid', 'cov']:
    train_dataset_path = f"../datasets/{hparams.dataset_name}_training.csv"
    test_dataset_path  = f"../datasets/{hparams.dataset_name}_testing.csv"

    train_test_split(X=np.loadtxt(dataset_path, delimiter=",", dtype=np.float32),
                    split=hparams.train_test_split,
                    train_file_name=train_dataset_path,
                    test_file_name=test_dataset_path    
                    )
    
    print(f"The {hparams.dataset_name} dataset has been split successfully into:\n\t- {train_dataset_path}\n\t- {test_dataset_path}")
elif hparams.dataset_name == 'real':
    train_dataset_path = hparams.train_file_path
    test_dataset_path  = hparams.test_file_path
else:
  raise ValueError("Dataset not supported.")

## 1.6 Model

This cell loads the TimeGAN model class.

In [ ]:
from timegan_model import TimeGAN

## 2 Train

This chapter will train the model according to the hyper-parameters defined above in section [Hyper-parameters](#13-hyper-parameters).

In [ ]:
accelerator = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Instantiate the model
timegan = TimeGAN(hparams=hparams,
                    train_file_path=train_dataset_path,
                    test_file_path=test_dataset_path
                    )

In [ ]:
# Define the logger -> https://www.wandb.com/articles/pytorch-lightning-with-weights-biases.
wandb_logger = WandbLogger(project="TimeGAN PyTorch (2024)", log_model=True)
wandb_logger.experiment.watch(timegan, log='all', log_freq=100)

In [ ]:
# Define the trainer
early_stop = EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=hparams.early_stop_patience,
    strict=False,
    verbose=False
)
trainer = Trainer(logger=wandb_logger,
                max_epochs=hparams.n_epochs,
                accelerator=accelerator,
                val_check_interval=0.25,
                callbacks=[early_stop]
                )

In [ ]:
# Start the training
trainer.fit(timegan)

In [ ]:
# Log the trained model
trainer.save_checkpoint('timegan.pth')
wandb.save('timegan.pth')

# 3 Validation

## 4 Visualize Results